In [1]:
import os

os.chdir("../")

In [2]:
from alm.lyrics import *
from alm.melody import *
from alm.comparator import *
from alm.utils import io
import pprint

In [3]:
input_list = [
    #[楽譜のXMLファイルのパス, タイムスパン木のXMLファイルのパス, セクション名]
    ["tests/files/popular/キセキ_A1.xml", "tests/files/popular/キセキ_A1_TS.xml", "キセキ_A1"],
    ["tests/files/popular/キセキ_A2.xml", "tests/files/popular/キセキ_A2_TS.xml", "キセキ_A2"],
    ["tests/files/popular/キセキ_S1.xml", "tests/files/popular/キセキ_S1_TS.xml", "キセキ_S1"],
    ["tests/files/popular/キセキ_S2.xml", "tests/files/popular/キセキ_S2_TS.xml", "キセキ_S2"],
    ["tests/files/popular/キセキ_S3.xml", "tests/files/popular/キセキ_S3_TS.xml", "キセキ_S3"],
    ["tests/files/popular/愛唄_A1.xml", "tests/files/popular/愛唄_A1_TS.xml", "愛唄_A1"],
    ["tests/files/popular/愛唄_A2.xml", "tests/files/popular/愛唄_A2_TS.xml", "愛唄_A2"],
    ["tests/files/popular/愛唄_S1.xml", "tests/files/popular/愛唄_S1_TS.xml", "愛唄_S1"],
    ["tests/files/popular/愛唄_S2.xml", "tests/files/popular/愛唄_S2_TS.xml", "愛唄_S2"],
    ["tests/files/popular/花唄_A1.xml", "tests/files/popular/花唄_A1_TS.xml", "花唄_A1"],
    ["tests/files/popular/花唄_A2.xml", "tests/files/popular/花唄_A2_TS.xml", "花唄_A2"],
    ["tests/files/popular/花唄_S1.xml", "tests/files/popular/花唄_S1_TS.xml", "花唄_S1"],
    ["tests/files/popular/花唄_S2.xml", "tests/files/popular/花唄_S2_TS.xml", "花唄_S2"],
    ["tests/files/popular/オレンジ_A1.xml", "tests/files/popular/オレンジ_A1_TS.xml", "オレンジ_A1"],
    ["tests/files/popular/オレンジ_A2.xml", "tests/files/popular/オレンジ_A2_TS.xml", "オレンジ_A2"],
    ["tests/files/popular/オレンジ_S1.xml", "tests/files/popular/オレンジ_S1_TS.xml", "オレンジ_S1"],
    ["tests/files/popular/オレンジ_S2.xml", "tests/files/popular/オレンジ_S2_TS.xml", "オレンジ_S2"],
    ["tests/files/popular/オレンジ_S3.xml", "tests/files/popular/オレンジ_S3_TS.xml", "オレンジ_S3"],
    ["tests/files/popular/オレンジ_S4.xml", "tests/files/popular/オレンジ_S4_TS.xml", "オレンジ_S4"],
    ["tests/files/popular/遥か_A1.xml", "tests/files/popular/遥か_A1_TS.xml", "遥か_A1"],
    ["tests/files/popular/遥か_A2.xml", "tests/files/popular/遥か_A2_TS.xml", "遥か_A2"],
    ["tests/files/popular/遥か_A3.xml", "tests/files/popular/遥か_A3_TS.xml", "遥か_A3"],
    ["tests/files/popular/遥か_A4.xml", "tests/files/popular/遥か_A4_TS.xml", "遥か_A4"],
    ["tests/files/popular/遥か_S1.xml", "tests/files/popular/遥か_S1_TS.xml", "遥か_S1"],
    ["tests/files/popular/遥か_S2.xml", "tests/files/popular/遥か_S2_TS.xml", "遥か_S2"],
    ["tests/files/unpopular/beautiful_days_A1.xml", "tests/files/unpopular/beautiful_days_A1_TS.xml", "beautiful_days_A1"],
    ["tests/files/unpopular/beautiful_days_A2.xml", "tests/files/unpopular/beautiful_days_A2_TS.xml", "beautiful_days_A2"],
    ["tests/files/unpopular/beautiful_days_S1.xml", "tests/files/unpopular/beautiful_days_S1_TS.xml", "beautiful_days_S1"],
    ["tests/files/unpopular/beautiful_days_S2.xml", "tests/files/unpopular/beautiful_days_S2_TS.xml", "beautiful_days_S2"],
    ["tests/files/unpopular/ビリーヴ_A1.xml", "tests/files/unpopular/ビリーヴ_A1_TS.xml", "ビリーヴ_A1"],
    ["tests/files/unpopular/ビリーヴ_A2.xml", "tests/files/unpopular/ビリーヴ_A2_TS.xml", "ビリーヴ_A2"],
    ["tests/files/unpopular/ビリーヴ_S1.xml", "tests/files/unpopular/ビリーヴ_S1_TS.xml", "ビリーヴ_S1"],
    ["tests/files/unpopular/ビリーヴ_S2.xml", "tests/files/unpopular/ビリーヴ_S2_TS.xml", "ビリーヴ_S2"],
    ["tests/files/unpopular/たけてん_A1.xml", "tests/files/unpopular/たけてん_A1_TS.xml", "たけてん_A1"],
    ["tests/files/unpopular/たけてん_A2.xml", "tests/files/unpopular/たけてん_A2_TS.xml", "たけてん_A2"],
    ["tests/files/unpopular/たけてん_S1.xml", "tests/files/unpopular/たけてん_S1_TS.xml", "たけてん_S1"],
    ["tests/files/unpopular/たけてん_S2.xml", "tests/files/unpopular/たけてん_S2_TS.xml", "たけてん_S2"],
    ["tests/files/unpopular/人_A1.xml", "tests/files/unpopular/人_A1_TS.xml", "人_A1"],
    ["tests/files/unpopular/人_A2.xml", "tests/files/unpopular/人_A2_TS.xml", "人_A2"],
    ["tests/files/unpopular/人_S1.xml", "tests/files/unpopular/人_S1_TS.xml", "人_S1"],
    ["tests/files/unpopular/人_S2.xml", "tests/files/unpopular/人_S2_TS.xml", "人_S2"],
    ["tests/files/unpopular/蕾_S1.xml", "tests/files/unpopular/蕾_S1_TS.xml", "蕾_S1"],
    ["tests/files/unpopular/蕾_S2.xml", "tests/files/unpopular/蕾_S2_TS.xml", "蕾_S2"],
    ["tests/files/unpopular/蕾_S3.xml", "tests/files/unpopular/蕾_S3_TS.xml", "蕾_S3"],
]

In [4]:
def calc_word_match_rate(input_list):
    parser = grammar_parser.GrammarParser("ja_ginza")

    res = []

    for ele in input_list:
        lyrics_notes_map = lyrics_extractor.extract_lyrics(ele[0])
        doc = parser.parse(lyrics_notes_map[lyrics_extractor.LYRICS_KEY])
        lyrics_tree = parser.to_tree(doc)
        words_notes_map = {}
        associating_lyrics_melody.explore_words_in_tree(lyrics_tree, words_notes_map)
        words_list = associating_lyrics_melody.associate_word_list_notes(words_notes_map, lyrics_notes_map)

        melody_tree = time_span_tree.time_span_tree_to_dict(ele[1])

        rate = word_match_rate_calculator.calc_word_match_rate(words_list, melody_tree)
        rate .section_name = ele[2]

        res.append([rate.section_name, rate.words_number, rate.match_words_number, rate.words_number - rate.match_words_number, rate.match_rate])
    
    return res

In [5]:
res = calc_word_match_rate(input_list)
io.output_csv("./notebooks/word_match_rate.csv", ["セクション名", "単語数", "一致した単語数", "一致しなかった単語数", "一致率"], res)
pprint.pprint(res)

[['キセキ_A1', 12, 8, 4, 0.6666666666666666],
 ['キセキ_A2', 13, 8, 5, 0.6153846153846154],
 ['キセキ_S1', 13, 9, 4, 0.6923076923076923],
 ['キセキ_S2', 12, 10, 2, 0.8333333333333334],
 ['キセキ_S3', 17, 11, 6, 0.6470588235294118],
 ['愛唄_A1', 18, 11, 7, 0.6111111111111112],
 ['愛唄_A2', 21, 16, 5, 0.7619047619047619],
 ['愛唄_S1', 15, 7, 8, 0.4666666666666667],
 ['愛唄_S2', 14, 7, 7, 0.5],
 ['花唄_A1', 22, 15, 7, 0.6818181818181818],
 ['花唄_A2', 20, 15, 5, 0.75],
 ['花唄_S1', 15, 12, 3, 0.8],
 ['花唄_S2', 14, 12, 2, 0.8571428571428571],
 ['オレンジ_A1', 11, 10, 1, 0.9090909090909091],
 ['オレンジ_A2', 22, 16, 6, 0.7272727272727273],
 ['オレンジ_S1', 11, 5, 6, 0.45454545454545453],
 ['オレンジ_S2', 15, 11, 4, 0.7333333333333333],
 ['オレンジ_S3', 22, 16, 6, 0.7272727272727273],
 ['オレンジ_S4', 11, 10, 1, 0.9090909090909091],
 ['遥か_A1', 6, 4, 2, 0.6666666666666666],
 ['遥か_A2', 11, 5, 6, 0.45454545454545453],
 ['遥か_A3', 12, 11, 1, 0.9166666666666666],
 ['遥か_A4', 10, 7, 3, 0.7],
 ['遥か_S1', 18, 16, 2, 0.8888888888888888],
 ['遥か_S2', 12, 8, 